In [1]:
!pip install datasets torch transformers matplotlib numpy tqdm

In [2]:
from google.colab import drive
drive.mount("/content/drive")

# PROJECT = "/content/drive/MyDrive/DLFinalProject/rank-collapse"
PROJECT = "/content/drive/MyDrive/rank-collapse"
%cd {PROJECT}

from utils import *

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1dwgmS-Fk3Kgg-RWfFuIvr8z2CnGXc00h/rank-collapse
🖥️  Using device: cuda
   GPU: NVIDIA A100-SXM4-40GB
✅ Setup complete!


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004-2d5a1467fff108(…):   0%|          | 0.00/249M [00:00<?, ?B/s]

data/train-00001-of-00004-5852b56a2bd28f(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/train-00002-of-00004-a26307300439e9(…):   0%|          | 0.00/246M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

✅ Datasets defined:
   - Copy (easy, diagonal attention)
   - Reverse (medium, anti-diagonal)
   - Sort (hard, global reasoning)
   - Text (language modeling)
   - Image (classification)
✅ Transformer architectures defined:
   - TINY: 2 layers, 64d, 4 heads (~0K params)
   - SMALL: 4 layers, 128d, 8 heads (~4K params)
   - MEDIUM: 6 layers, 256d, 8 heads (~12K params)
   - LARGE: 8 layers, 512d, 16 heads (~65K params)
✅ Rank metrics defined:
   - Stable rank (Frobenius / Operator norm)
   - Entropy effective rank
   - Top-k energy fraction
   - Attention sparsity
✅ Training system ready!
✅ Visualization tools ready!


In [3]:
def run_collapse_expand_combos(
        task_name="copy", dataset_size_name="medium", arch="small",
        collapse_func=None, expand_func=None, steps=2000,
        batch_size=64, out_dir="logs/weight0.01", seed=42):
    """
    Runs:
      - collapse early (first25) then expand late (last25)
      - expand early then collapse late
      - collapse early only
      - expand early only
    Uses collapse_func and expand_func from dictionaries.
    """
    combos = [
        ("collapseEarlyThenExpandLate", [ (0, int(0.25*steps)-1, collapse_func), (int(0.75*steps), steps-1, expand_func) ]),
        ("expandEarlyThenCollapseLate", [ (0, int(0.25*steps)-1, expand_func), (int(0.75*steps), steps-1, collapse_func) ]),
        ("collapseEarlyOnly", [ (0, int(0.25*steps)-1, collapse_func) ]),
        ("expandEarlyOnly", [ (0, int(0.25*steps)-1, expand_func) ]),
    ]
    sizes = {"small": 1024, "medium": 4096, "large": 8192}

    for combo_name, funcs in combos:
        print(f"RUN combo {combo_name}")
        # build TrainingScheduler from funcs mapping to (start,end,func)
        scheduler = TrainingScheduler(which_layers="all", base_func=cross_entropy, functions_at_times=funcs)
        # build datasets & model
        dataset_size = sizes[dataset_size_name]
        ds_train, task_type, vocab_size, _ = create_dataset(
            task_name, seq_len=32, vocab_size=64, dataset_size=dataset_size,
            split="train")
        ds_probe, _, _, _ = create_dataset(
            task_name, seq_len=32, vocab_size=64,
            dataset_size=min(512, dataset_size), split="train")
        model, arch_config = instantiate_architecture(arch, vocab_size=max(32, vocab_size), seq_len=32, task_type=task_type)
        model = model.to(device)

        # wrapper loss to ensure scheduled functions get model/layers/activations
        # we reuse run_experiment but pass scheduler object directly: run_experiment calls scheduler.call_function with **kwargs
        log_name = f"{task_name}_{dataset_size_name}_{arch}_{combo_name}_steps{steps}_seed{seed}.npy"
        out_path = os.path.join(out_dir, log_name)
        os.makedirs(out_dir, exist_ok=True)

        if not os.path.exists(out_path):
            model, _ = run_experiment((ds_train, task_type, None, None),
                (ds_probe, task_type, None, None),
                model, arch_config,
                steps=steps, batch_size=batch_size,
                checkpoint_every=max(1, steps//10), lr=3e-4,
                seed=seed, loss_scheduler=scheduler, metric_func=full_spectrum_metrics,
                log_store_name=out_path, weight_decay=0.01)
            print(f"Saved combo log to {out_path}")
        else:
            print(f"{out_path} already done!")

def run_collapse_expand_sweeps(
        collapse_func, expand_func, tasks=None, archs=None,
        dataset_sizes=None, steps=2000, batch_size=64,
        out_dir="logs", seed=42):
    """
    Iterate over tasks × archs × dataset sizes × regs × timings and run experiments.
    By default uses the increase_dict keys then decrease_dict keys.
    """
    if tasks is None:
        tasks = ["copy", "sort", "tiny_stories"]
    if archs is None:
        archs = ["small", "medium", "large"]
    if dataset_sizes is None:
        dataset_sizes = ["medium"]

    all_runs = []

    for task in tasks:
        for size_name in dataset_sizes:
            for arch in archs:
                print(f"RUN: task={task} size={size_name} arch={arch}")
                run_collapse_expand_combos(
                    task, size_name, arch, collapse_func, expand_func, steps,
                    batch_size, out_dir=out_dir, seed=seed)
    return all_runs

In [4]:
def run_tests(weight=1.0):
    collapse_funcs = ["logdet", "erank"]
    expand_funcs = ["orthonorm", "spectral_norm"]

    for collapse_name in collapse_funcs:
        for expand_name in expand_funcs:
            if collapse_funcs == "erank" and expand_funcs == "spectral_norm":
                return
            collapse_func = lambda logits, yb, **kwargs: weight * increase_rank_regularizers[collapse_name](logits, yb, **kwargs)
            expand_func = decrease_rank_regularizers[expand_name]
            out_dir = f"logs/{weight}/{collapse_name}_{expand_name}"
            run_collapse_expand_sweeps(collapse_func, expand_func, out_dir=out_dir)

In [ ]:
run_tests(1.0)

RUN: task=copy size=medium arch=small
RUN combo collapseEarlyThenExpandLate
logs/1.0/logdet_orthonorm/copy_medium_small_collapseEarlyThenExpandLate_steps2000_seed42.npy already done!
RUN combo expandEarlyThenCollapseLate
logs/1.0/logdet_orthonorm/copy_medium_small_expandEarlyThenCollapseLate_steps2000_seed42.npy already done!
RUN combo collapseEarlyOnly
logs/1.0/logdet_orthonorm/copy_medium_small_collapseEarlyOnly_steps2000_seed42.npy already done!
RUN combo expandEarlyOnly
logs/1.0/logdet_orthonorm/copy_medium_small_expandEarlyOnly_steps2000_seed42.npy already done!
RUN: task=copy size=medium arch=medium
RUN combo collapseEarlyThenExpandLate
logs/1.0/logdet_orthonorm/copy_medium_medium_collapseEarlyThenExpandLate_steps2000_seed42.npy already done!
RUN combo expandEarlyThenCollapseLate
logs/1.0/logdet_orthonorm/copy_medium_medium_expandEarlyThenCollapseLate_steps2000_seed42.npy already done!
RUN combo collapseEarlyOnly
logs/1.0/logdet_orthonorm/copy_medium_medium_collapseEarlyOnly_step

Training: 100%|██████████| 2000/2000 [03:06<00:00, 10.74step/s, loss=0.0048, acc=1.0000]


✅ Training complete in 186.2s
Saved combo log to logs/1.0/logdet_orthonorm/copy_medium_large_collapseEarlyOnly_steps2000_seed42.npy
RUN combo expandEarlyOnly
Model: 8 layers, 512d, 16 heads
Parameters: 25,547,840
Device: cuda


Training: 100%|██████████| 2000/2000 [02:56<00:00, 11.35step/s, loss=0.0000, acc=1.0000]


✅ Training complete in 176.2s
Saved combo log to logs/1.0/logdet_orthonorm/copy_medium_large_expandEarlyOnly_steps2000_seed42.npy
RUN: task=sort size=medium arch=small
RUN combo collapseEarlyThenExpandLate
Model: 4 layers, 128d, 8 heads
Parameters: 875,328
Device: cuda


...

Training: 100%|██████████| 2000/2000 [02:22<00:00, 14.01step/s, loss=0.0001, acc=1.0000]


✅ Training complete in 142.7s
Saved combo log to logs/1.0/logdet_spectral_norm/copy_medium_medium_expandEarlyOnly_steps2000_seed42.npy
RUN: task=copy size=medium arch=large
RUN combo collapseEarlyThenExpandLate
Model: 8 layers, 512d, 16 heads
Parameters: 25,547,840
Device: cuda


Training: 100%|██████████| 2000/2000 [06:58<00:00,  4.78step/s, loss=0.0023, acc=1.0000]


✅ Training complete in 418.8s
Saved combo log to logs/1.0/logdet_spectral_norm/copy_medium_large_collapseEarlyThenExpandLate_steps2000_seed42.npy
RUN combo expandEarlyThenCollapseLate
Model: 8 layers, 512d, 16 heads
Parameters: 25,547,840
Device: cuda


Training:   3%|▎         | 65/2000 [00:44<15:31,  2.08step/s, loss=14.4380]